## Running the 2x2 Simulation chain on Polaris using Singularity container

In [ ]:
#Obtain a debug node to do our work for all the steps below
qsub -I -l select=1 -l filesystems=home:eagle -l walltime=1:00:00 -q debug -A ALCF_for_DUNE

In [ ]:
# Module load command for software loading
module use /soft/modulefiles
module load spack-pe-base/0.6.1
module load apptainer

#donot load python as it messes with python in the container#or maybe try loading python if it doesn't work
module use /soft/modulefiles ; module load conda; conda activate base


In [ ]:
export ARCUBE_RUNTIME='SINGULARITY'
export ARCUBE_CONTAINER_DIR='/grand/ALCF_for_DUNE/2x2_sim_data/containers'

#Change this to your directory
ARCUBE_DIR='/home/aleena/2x2_sim_111424/2x2_sim'

#change the directory to where you want the output files
export ARCUBE_OUTDIR_BASE='/grand/ALCF_for_DUNE/users/aleena/2x2_sim_111424/out'
export ARCUBE_LOGDIR_BASE='/grand/ALCF_for_DUNE/users/aleena/2x2_sim_111424/log'


In [ ]:

#install mlreco
cd run-mlreco


export ARCUBE_CONTAINER='larcv2_ub20.04-cuda11.6-pytorch1.13-larndsim.sif'
export ARCUBE_RUNTIME='SINGULARITY'
export SINGULARITY_BIND=$ARCUBE_DIR,/grand/ALCF_for_DUNE:/grand/ALCF_for_DUNE

#edit install file
'''
-export ARCUBE_RUNTIME=SHIFTER
+#export ARCUBE_RUNTIME=SHIFTER
 # export ARCUBE_CONTAINER=deeplearnphysics/larcv2:ub20.04-cuda11.3-cudnn8-pytorch1.10.0-larndsim-2022-11-03
 ## Above but with venv support:
 # export ARCUBE_CONTAINER=mjkramer/sim2x2:mlreco001
 # This is the one that Francois has actually been using:
-export ARCUBE_CONTAINER=deeplearnphysics/larcv2:ub20.04-cuda11.6-pytorch1.13-larndsim
+#export ARCUBE_CONTAINER=deeplearnphysics/larcv2:ub20.04-cuda11.6-pytorch1.13-larndsim
 
 source ../util/reload_in_container.inc.sh
 
@@ -13,6 +13,8 @@ set -o errexit
 
 # recommended by Jeremy
 unset which
+unset LD_PRELOAD
+
#edit util/reload_in_container.inc.sh file
add --nv to the singularity exec line (singularity exec --nv -B $ARCUBE_DIR ...)
'''

#I see that when we install mlreco, we get weird error related to the container, therefore better is to open the install file and copy commands one-by-one and run them on the terminal
#This is the error I am getting: FATAL:   While checking container encryption: could not open image /bin/bash: image format not recognized.
./install_mlreco.sh 

In [ ]:
#install caf
cd run-cafmaker

#edit install file to look like this (COPY PASTE THIS AS THE NEW INSTALL SCRIPT)
'''
#!/usr/bin/env bash

#export ARCUBE_RUNTIME=SHIFTER
#export ARCUBE_CONTAINER=fermilab/fnal-wn-sl7:latest

source ../util/reload_in_container.inc.sh

source /cvmfs/dune.opensciencegrid.org/products/dune/setup_dune.sh

setup dune_oslibs v1_0_0
setup edepsim v3_2_0c -q e20:prof

#mkdir install
#cd install

#git clone -b v4.7.0 https://github.com/DUNE/ND_CAFMaker.git
cd install/ND_CAFMaker
sed -i 's/CXXFLAGS = /CXXFLAGS = -DDISABLE_TMS /' Makefile
./build.sh
source ndcaf_setup.sh
make -j8
'''

mkdir install
cd install
git clone -b v4.7.0 https://github.com/DUNE/ND_CAFMaker.git
cd ..
git clone https://github.com/cvmfs/cvmfsexec
cd cvmfsexec/
./makedist osg
cd ..

#load apptainer
module use /soft/modulefiles
module load spack-pe-base/0.6.1
module load apptainer
export ARCUBE_CONTAINER='fnal-wn-sl7_1.0.sif'
export ARCUBE_RUNTIME='SINGULARITY'

### !! edit this to your path
export CVMFS_PATH='/home/aleena/2x2_sim_111424/2x2_sim/run-cafmaker/cvmfsexec/'

export SINGULARITY_BIND=$ARCUBE_DIR,/grand/ALCF_for_DUNE:/grand/ALCF_for_DUNE,$CVMFS_PATH/dist/cvmfs:/cvmfs

./cvmfsexec/cvmfsexec config-osg.opensciencegrid.org dune.opensciencegrid.org fermilab.opensciencegrid.org larsoft.opensciencegrid.org dune.osgstorage.org -- ./install_cafmaker.sh

In [ ]:
'''
minerva
'''

### !! edit this to your path
export CVMFS_PATH='/home/fathimamaha/test2/2x2_sim/run-cafmaker/cvmfsexec'
export SINGULARITY_BIND=$ARCUBE_DIR,/grand/ALCF_for_DUNE:/grand/ALCF_for_DUNE,$CVMFS_PATH/dist/cvmfs:/cvmfs


cd run-edep2flat

export ARCUBE_CONTAINER='fnal-wn-sl7_1.0.sif'
export ARCUBE_RUNTIME='SINGULARITY'
export ARCUBE_IN_NAME='Tutorial.spill'
export ARCUBE_OUT_NAME='Tutorial.edep2flat'
export ARCUBE_INDEX=0

../run-cafmaker/cvmfsexec/cvmfsexec config-osg.opensciencegrid.org dune.opensciencegrid.org fermilab.opensciencegrid.org larsoft.opensciencegrid.org dune.osgstorage.org -- ./run_edep2flat.sh

cd run-minerva

export ARCUBE_CONTAINER='fnal-wn-sl7_1.0.sif'
export ARCUBE_RUNTIME='SINGULARITY'
export ARCUBE_IN_NAME='Tutorial.edep2flat'
export ARCUBE_OUT_NAME='Tutorial.minerva'
export ARCUBE_INDEX=0

../run-cafmaker/cvmfsexec/cvmfsexec config-osg.opensciencegrid.org dune.opensciencegrid.org fermilab.opensciencegrid.org larsoft.opensciencegrid.org dune.osgstorage.org minerva.opensciencegrid.org -- ./run_minerva.sh



In [ ]:

'''
mlreco
'''

export SINGULARITY_BIND=$ARCUBE_DIR,/grand/ALCF_for_DUNE:/grand/ALCF_for_DUNE

export ARCUBE_CONTAINER='larcv2_ub20.04-cuda11.6-pytorch1.13-larndsim.sif'
export ARCUBE_RUNTIME='SINGULARITY'
export ARCUBE_IN_NAME='flow'
export ARCUBE_OUT_NAME='flow2supera'
export ARCUBE_INDEX=0

# flow2supera stage takes an extremely long time to run. It is better to run it with nohup if you are on the login node.
./run_flow2supera.sh


export ARCUBE_IN_NAME='flow2supera'
export ARCUBE_OUT_NAME='spine'
export ARCUBE_INDEX=0

./run_spine.sh

# Old flow
export ARCUBE_IN_NAME='flow2supera'
export ARCUBE_OUT_NAME='mlreco_inference'
export ARCUBE_INDEX=0

./run_mlreco_inference.sh

export ARCUBE_IN_NAME='mlreco_inference'
export ARCUBE_OUT_NAME='mlreco_analysis'
export ARCUBE_INDEX=0

./run_mlreco_analysis.sh

In [ ]:

cd run-cafmaker

# !! SINCE WE ONLY PRODUCED ONE FILE with HADD_FACTOR=1, CHANGE run-cafmaker/./gen_cafmaker_cfg.py HADD_FACTOR=1


# add unset LD_PRELOAD and setup dune_oslibs v1_0_0 in run_cafmaker.sh
'''
 cd install/ND_CAFMaker
 source ndcaf_setup.sh
 cd ../..
 
+unset LD_PRELOAD
+setup dune_oslibs v1_0_0
+

'''


### !! edit this to your path
export CVMFS_PATH='/home/aleena/2x2_sim_111424/2x2_sim/run-cafmaker/cvmfsexec'
export SINGULARITY_BIND=$ARCUBE_DIR,/grand/ALCF_for_DUNE:/grand/ALCF_for_DUNE,$CVMFS_PATH/dist/cvmfs:/cvmfs


module use /soft/modulefiles
module load spack-pe-base/0.6.1
module load apptainer
export ARCUBE_CONTAINER='fnal-wn-sl7_1.0.sif'
export ARCUBE_RUNTIME='SINGULARITY'
export ARCUBE_GHEP_NU_NAME='genie.nu'
export ARCUBE_GHEP_ROCK_NAME='genie.rock'
export ARCUBE_SPILL_NAME='spill'
export ARCUBE_MINERVA_NAME='minerva'
export ARCUBE_SPINE_NAME='spine'
export ARCUBE_OUT_NAME='caf'
export ARCUBE_INDEX=0

./cvmfsexec/cvmfsexec config-osg.opensciencegrid.org dune.opensciencegrid.org fermilab.opensciencegrid.org larsoft.opensciencegrid.org dune.osgstorage.org -- ./run_cafmaker.sh

##fardetector same as this